In [4]:
from IPython.display import clear_output
import ipywidgets as wdg #This library is used to create functional interactive controls on the graphs.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #This library is used to plot the graphs.
import json #This library is used to create and store the date files. 
from uk_covid19 import Cov19API
#This cell imports all the relevant libraries that will be used in this dashboard. 

In [5]:
%matplotlib inline 
#This code prints the results of matplotlib in this notebook rather than a new window.

plt.rcParams['figure.dpi'] = 125 # make figures larger, this can be changed by amending the number '100'

In [6]:
# This loads JSON files which contain raw data from the API and store the raw data in a variable, jsondata.
with open("timeseries_1.json", "rt") as INFILE_1:
    data_1=json.load(INFILE_1)
 # this  data relates to female and males in England,specifically, case rates. I would have done the whole jurisdiction i.e. nation but this returns too much data and the error IOPub data rate exceeded occurs.
with open("timeseries_2.json", "rt") as INFILE_2:
    data_2=json.load(INFILE_2)
#this data relates to female and males in England, specifically regarding the number of deaths for each gender. 
with open("timeseries_3.json", "rt") as INFILE_3:
    data_3=json.load(INFILE_3)
#this data relates to the number of cases, hostipal admissions and deaths in all nations.
with open("timeseries_4.json", "rt") as INFILE_4:
    data_4=json.load(INFILE_4)
#this data relates to the number of current cases, deaths , those who have been vacinnated with the first and second doeses. 
with open("timeseries_5.json", "rt") as INFILE_5:
    data_5=json.load(INFILE_5)
#this data relates to cases, admissions to hospital, deaths and vaccinations regionally for London.
with open("timeseries_6.json", "rt") as INFILE_6:
    data_6=json.load(INFILE_6)
#this data relates to the number of cases, hostipal admissions and deaths in all nations .
jsondata={
'timeseries_1': data_1,
'timeseries_2': data_2,
'timeseries_3': data_3,
'timeseries_4': data_4,
'timeseries_5': data_5,
'timeseries_6': data_6
}



In [7]:
def wrangle_data_1 (data_1):# data_1 is the parameters and can either be the json file or api call which will return a dateframe. By creating a function, it allows you to call it again after refreshing the data through the API.
    data_dic= jsondata['timeseries_1']['data'][0]
    males_data= data_dic['males']
    females_data=data_dic ['females']
    males_data
    females_data
    ageranges_males_data_1=[x['age'] for x in males_data] 
    def min_age_gender_cases(ageranges_males_data_1):
        ageranges_males_data_1=ageranges_males_data_1.replace('+','') # This will remove any +'s from the categories. 
        start=ageranges_males_data_1.split('_')[0]
        return (int(start))

#We could also do this process for the age ranges of females from the data, however, as these age ranges are the same, there is little merit in doing this.
    ageranges_males_data_1.sort(key=min_age_gender_cases)
    df1=pd.DataFrame(index=ageranges_males_data_1, columns=['males','females', 'total'])
    for entry in males_data: 
        ageband=entry['age']
        df1.loc[ageband, 'males']=entry['value']
    for entry in females_data:
        ageband=entry['age']
        df1.loc[ageband, 'females']=entry['value']
    df1['total']=df1['males']+ df1['females'] #this combines the results in 'males' and 'females' to produce the total.
    #each entry is a seperate dictionary as this allows for the user to see three options in the widget, male, female and total.
    return df1
    


df1=wrangle_data_1(jsondata) #This produces the dataframe that is used to plot the graphs below. 

# The rest of the wrangle_datas follow a similar format but are curtailed to the filters and structures used. Wrangle_data_2 is very similar to wrangle_data_1. The only difference is wrangle_data_1 is for the collection of data of male and females cases whereas wrangle_data_2 collates data of male and female deaths.
def wrangle_data_2 (data_2):
    data_dic= jsondata['timeseries_2']['data'][0]
    males_deaths_data= data_dic['male_death']
    females_death_data=data_dic ['female_death']
    males_deaths_data
    females_death_data
    ageranges_males_data_2=[x['age'] for x in males_deaths_data] 
    def min_age_gender_death(ageranges_males_data_2):
        ageranges_males_data_2=ageranges_males_data_2.replace('+','') # This will remove any +'s from the categories. 
        start=ageranges_males_data_2.split('_')[0]
        return (int(start))
#We could also do this process for the age ranges of females from the data, however, as these age ranges are the same, there is little merit in doing this.
    ageranges_males_data_2.sort(key=min_age_gender_death)
    df2=pd.DataFrame(index=ageranges_males_data_2, columns=['male_death','female_death', 'total'])
    for entry in males_deaths_data: 
        ageband=entry['age']
        df2.loc[ageband, 'male_death']=entry['value']
    for entry in females_death_data:
        ageband=entry['age']
        df2.loc[ageband, 'female_death']=entry['value']
    df2['total']=df2['male_death']+ df2['female_death']
    #each entry is a seperate dictionary as this allows for the user to see three options in the widget, male, female and total.
    return df2

df2=wrangle_data_2(jsondata) #This produces the dataframe that is used to plot the graphs below. 


def wrangle_data_3(data_3):
    datalist_3= jsondata['timeseries_3']['data']
    datalist_3
    dates= [dictionary['date'] for dictionary in datalist_3]
    dates.sort()
    dates_3=dates
    def parse_date_3(datestring):
        return pd.to_datetime(datestring, format="%Y-%m-%d")
#this function will convert the dictionary inserted into the parameters needed for panda to work.
    start_date= parse_date_3 (dates_3[0])
    end_date= parse_date_3(dates_3[-1])
    index=pd.date_range(start_date, end_date, freq='D') #This scode creates the range of dates that form the index of the dataframe.
    
    df3=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths', 'cumulativedeaths'])
    for entry in datalist_3: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date_3(entry['date'])
        for column in ['cases', 'hospital', 'deaths', 'cumulativedeaths']:
# This for loop will check that nothing is in these columns yet.
            if pd.isna(df3.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0 # replace None with 0 in our data  
                df3.loc[date, column]=value # To access a specific location in the dataframe - use .loc       
# fill in any remaining "holes" due to missing dates
    df3.fillna(0.0, inplace=True)
    return df3

df3=wrangle_data_3(jsondata) #This produces the dataframe that is used to plot the graphs below.  


# Wrangle_data_4-6 is essentially based on wrangle_data_3 but edited to suit the filters and structures of the different graphs.
def wrangle_data_4(data_4):
    datalist_4= jsondata ['timeseries_4']['data']
    dates=[dictionary['date'] for dictionary in datalist_4]
    dates.sort()
    dates_4=dates
    def parse_date_4(datestring):
        return pd.to_datetime(datestring, format="%Y-%m-%d")
    start_date= parse_date_4 (dates_4[0])
    end_date= parse_date_4(dates_4[-1])
    index_4=pd.date_range(start_date, end_date, freq='D')
    df4=pd.DataFrame(index=index_4, columns=['cases', 'hospital', 'deaths', 'No_of_people_vaccinated_1st_dose', 'No_of_people_vaccinated_2nd_dose'])
    for entry in datalist_4: 
        date=parse_date_4(entry['date'])
        for column in ['cases', 'hospital', 'deaths', 'No_of_people_vaccinated_1st_dose', 'No_of_people_vaccinated_2nd_dose']:
            if pd.isna(df4.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                df4.loc[date, column]=value
    df4.fillna(0.0, inplace=True)
    return df4
    
df4=wrangle_data_4(jsondata) #This produces the dataframe that is used to plot the graphs below.


def wrangle_data_5(data_5):
    datalist_5= jsondata ['timeseries_5']['data']
    dates=[dictionary['date'] for dictionary in datalist_5]
    dates.sort()
    dates_5=dates
    def parse_date_5(datestring):
        return pd.to_datetime(datestring, format="%Y-%m-%d")
    start_date= parse_date_5 (dates_5[0])
    end_date= parse_date_5(dates_5[-1])
    index_5=pd.date_range(start_date, end_date, freq='D')
    df5=pd.DataFrame(index=index_5, columns=['cases', 'hospital', 'deaths', 'No_of_people_vaccinated_1st_dose', 'No_of_people_vaccinated_2nd_dose'])
    
    for entry in datalist_5: 
        date=parse_date_5(entry['date'])
        for column in ['cases', 'hospital', 'deaths', 'No_of_people_vaccinated_1st_dose', 'No_of_people_vaccinated_2nd_dose']:
            if pd.isna(df5.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                df5.loc[date, column]=value
    df5.fillna(0.0, inplace=True)
    return df5 #This produces the dataframe that is used to plot the graphs below.

df5=wrangle_data_5(jsondata) #This produces the dataframe that is used to plot the graphs below.


def wrangle_data_6(data_6):
    datalist_6= jsondata ['timeseries_6']['data']
    dates=[dictionary['date'] for dictionary in datalist_6]
    dates.sort()
    dates_6=dates
    def parse_date_6(datestring):
        return pd.to_datetime(datestring, format="%Y-%m-%d")
    start_date_6= parse_date_6(dates_6[0])
    end_date_6= parse_date_6(dates_6[-1])
    index_6=pd.date_range(start_date_6, end_date_6, freq='D')
    df6=pd.DataFrame(index=index_6, columns=['cases', 'hospital', 'occupied_ventilators', 'deaths'])
    for entry in datalist_6: 
        date=parse_date_6(entry['date'])
        for column in ['cases', 'hospital', 'occupied_ventilators', 'deaths']:
            if pd.isna(df6.loc[date, column]): 
            # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                df6.loc[date, column]=value
            
    df6.fillna(0.0, inplace=True)
    return df6

df6=wrangle_data_6(jsondata) #This produces the dataframe that is used to plot the graphs below.


In [8]:
def access_api(): #The next two cells create the refresh data button which contacts the api and wrangles the new data into the aforementioned dataframes.
    filter_1= [
    'areaType=nation','areaName=England'
]
    structure_1 = {
    "males": "maleCases",
    "females": "femaleCases",
    }
    api_1 = Cov19API(filters=filter_1, structure=structure_1)
    timeseries_1=api_1.get_json()
    
    filter_2= [
    'areaType=nation','areaName=England'
    ]
    structure_2 = {
    "male_death": "maleDeaths28Days",
    "female_death": "femaleDeaths28Days"
    }
    api_2 = Cov19API(filters=filter_2, structure=structure_2)
    timeseries_2=api_2.get_json()
    
    filters_3 = ['areaType=nation']
    structure_3 = {
            "date": "date",
            "cases": "newCasesBySpecimenDateRollingRate",
            "hospital": "newAdmissionsRollingRate",
            "deaths": "newDailyNsoDeathsByDeathDate",
            "cumulativedeaths": "cumDailyNsoDeathsByDeathDate"
        }
    api_3 = Cov19API(filters=filters_3, structure=structure_3)
    timeseries_3 = api_3.get_json()
    
    filters_4 = ['areaType=nation']
    structure_4 = {
        "date": "date",
        "cases": "newCasesBySpecimenDateRollingRate",
        "hospital": "newAdmissionsRollingRate",
        "deaths": "newDailyNsoDeathsByDeathDate", 
        "No_of_people_vaccinated_1st_dose": "cumPeopleVaccinatedFirstDoseByPublishDate",
        "No_of_people_vaccinated_2nd_dose": "cumPeopleVaccinatedCompleteByPublishDate"
    }
    api_4 = Cov19API(filters=filters_4, structure=structure_4)
    timeseries_4 = api_4.get_json()

    filters_5 = ['areaType=region', 'areaName=London']
    structure_5 = {
        "date": "date",
        "cases": "newCasesBySpecimenDateRollingRate",
        "hospital": "newAdmissionsRollingRate",
        "deaths": "newDailyNsoDeathsByDeathDate", 
        "No_of_people_vaccinated_1st_dose": "cumPeopleVaccinatedFirstDoseByPublishDate",
        "No_of_people_vaccinated_2nd_dose": "cumPeopleVaccinatedCompleteByPublishDate"
    }
    api_5 = Cov19API(filters=filters_5, structure=structure_5)
    timeseries_5 = api_5.get_json()

    filters_6 = ['areaType=nation']
    structure_6 = {
        "date": "date",
        "cases": "newCasesBySpecimenDateRollingRate",
        "hospital": "newAdmissionsRollingRate",
        "occupied_ventilators": "covidOccupiedMVBeds",
        "deaths": "newDailyNsoDeathsByDeathDate"
    }
    api_6 = Cov19API(filters=filters_6, structure=structure_6)
    timeseries_6 = api_6.get_json()# these filters and structures are used to gather the relevant data for the wrangling in the cell below. 
    return timeseries_1, timeseries_2, timeseries_3, timeseries_4, timeseries_5, timeseries_6
    

In [9]:
def api_button_callback(button):
    #This assigns the data 
    apidata=access_api() 
    global df1
    df1=wrangle_data_1(apidata)
    global df2
    df2= wrangle_data_2(apidata)
    global df3
    df3=wrangle_data_3(apidata)
    global df4
    df4=wrangle_data_4(apidata)
    global df5
    df5=wrangle_data_5(apidata)
    global df6
    df6=wrangle_data_6(apidata)
    #The above lines of code access the global dataframes i.e. the dataframes plotted above and replots them.
    apibutton.icon="check"
    apibutton.disabled=True
    #These two lines ensures the button is disabled after being clicked on.
 
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)
# This is the code that creates the interactable button.

apibutton.on_click(api_button_callback)
# This registers the callback function with the button
display(apibutton)
#This code displays the button


Button(button_style='success', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Clic…

In [10]:
df1_interactive_series=wdg.SelectMultiple(
    options=['males', 'females', 'total'], 
    value=['males', 'females'], 
    rows=3,
    description='Gender',
    disabled=False
)

df1_interactive_scale=wdg.RadioButtons(
    options=['bar', 'linear'],
    description='Type of graph:',
    disabled=False
)

controls=wdg.VBox([df1_interactive_series, df1_interactive_scale])

def df1_interactive_graph(gcols, gscale):
    ncols = len(gcols)
  
    if gscale == 'bar':
        linearscale = False
        if ncols > 0:
            df1.plot(kind='bar', y=list(gcols))
            plt.show()
# The above line is important as graphs won't update properly if this is missing
        else:
            print("Please click one of the interactable buttons")
            print("If you would like to select more than one category, hold down tab and select with your mouse")
    else:
        linearscale = True
        if ncols > 0:
            df1[list(gcols)].plot()
            plt.show()
 # The above line is important as graphs won't update properly if this is missing
        else:
            print("Please click one of the interactable buttons")
            print("If you would like to select more than one category, hold down tab and select with your mouse")

graph_1 = wdg.interactive_output(df1_interactive_graph, {'gcols': df1_interactive_series, 'gscale': df1_interactive_scale})

#The above code creates multiple interactive controls on the graph.

def refresh_graph(button):
    current=df1_interactive_scale.value
    if current == df1_interactive_scale.options[0]:
        other= df1_interactive_scale.options[1]
    else:
        other= df1_interactive_scale.options[0]
    df1_interactive_scale.value=other
    df1_interactive_scale.value=current

#This code allows for the graph to refresh with the new data. This will only work properly if the refresh data is pressed first (see above) and then the refresh graph
refresh= wdg.Button(
    description='refresh graph',
    disabled=False,
    button_style='success',
    tooltip= 'click to refresh data in graph',
    icon= 'arrows'
)
#This code creates the interactable button 
#I decided to not make the button one click only as users may want to refresh data for a graph more than once.
refresh.on_click(refresh_graph)
# This registers the callback function with the button

display(controls,refresh, graph_1) 
#displays graph, refresh button and interactive controls.
#I decided to do widgets on all of my graphs therefore there is no simple plot function as the plot function is in-built into the code fulfilling both of the coursework requirements in that respect. 

#The below code is based on the above.
df2_interactive_series=wdg.SelectMultiple(
    options =['male_death', 'female_death', 'total'],
    value = ['male_death', 'female_death'],
    rows=3,
    description= 'deaths',
    disable=False)

df2_interactive_scale=wdg.RadioButtons(
    options=['bar', 'linear'],
    description='Type of graph:',
    disabled=False
)

controls=wdg.HBox([df2_interactive_series, df2_interactive_scale])

def df2_interactive_graph(gcols, gscale):
    ncols = len(gcols)
  
    if gscale == 'bar':
        linearscale = False
        if ncols > 0:
            df2.plot(kind='bar', y=list(gcols))
            plt.show() 
            # The above line is important as graphs won't update properly if this is missing
        else:
            print("Please click one of the interactable buttons")
            print("If you would like to select more than one category, hold down tab and select with your mouse")
    else:
        linearscale = True
        if ncols > 0:
            df2[list(gcols)].plot()
            plt.show()
             # The above line is important as graphs won't update properly if this is missing
        else:
            print("Please click one of the interactable buttons")
            print("If you would like to select more than one category, hold down tab and select with your mouse")

graph_2 = wdg.interactive_output(df2_interactive_graph, {'gcols': df2_interactive_series, 'gscale': df2_interactive_scale})

def refresh_graph(button):
    current=df2_interactive_scale.value
    if current == df2_interactive_scale.options[0]:
        other= df2_interactive_scale.options[1]
    else:
        other= df2_interactive_scale.options[0]
    df2_interactive_scale.value=other
    df2_interactive_scale.value=current

refresh= wdg.Button(
    description='refresh graph',
    disabled=False,
    button_style='success',
    tooltip= 'click to refresh data in graph',
    icon= 'arrows'
)
#I decided to not make the button one click only as users may want to refresh data for a graph more than once.
refresh.on_click(refresh_graph)

display(controls,refresh, graph_2)

df3_interactive_series=wdg.SelectMultiple(
    options =['cases', 'hospital','deaths', 'cumulativedeaths'],
    value = ['cases', 'hospital','deaths', 'cumulativedeaths'],
    rows=4,
    description= 'Category:',
    disable=False)

df3_interactive_scale=wdg.RadioButtons(
    options=['log', 'linear'],
#   value='pineapple', # Defaults to 'pineapple'
#   layout={'width': 'max-content'}, # If the items' names are long
    description='Type of graph:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.VBox([df3_interactive_series, df3_interactive_scale])

def df3_interactive_graph(gcols, gscale):
    ncols = len(gcols)
  
    if gscale == 'linear':
        logscale = False
        if ncols > 0:
            df3[list(gcols)].plot()
            plt.show()
        else:
            print("Please click one of the interactable buttons")
            print("If you would like to select more than one category, hold down tab and select with your mouse")
    else:
        logscale = True
        if ncols > 0:
            df3[list(gcols)].plot(logy=logscale)
            plt.show()
        else:
            print("Please click one of the interactable buttons")
            print("If you would like to select more than one category, hold down tab and select with your mouse")

graph_3 = wdg.interactive_output(df3_interactive_graph, {'gcols': df3_interactive_series, 'gscale': df3_interactive_scale})

def refresh_graph(button):
    current=df3_interactive_scale.value
    if current == df3_interactive_scale.options[0]:
        other= df3_interactive_scale.options[1]
    else:
        other= df3_interactive_scale.options[0]
    df3_interactive_scale.value=other
    df3_interactive_scale.value=current

refresh= wdg.Button(
    description='refresh data',
    disabled=False,
    button_style='success',
    tooltip= 'click to refresh data in graph',
    icon= 'arrows'
)
#I decided to not make the button one click only as users may want to refresh data for a graph more than once.
refresh.on_click(refresh_graph)

display(controls,refresh, graph_3)


df4_interactive_series=wdg.SelectMultiple(
    options =['cases', 'hospital','deaths', 'No_of_people_vaccinated_1st_dose', 'No_of_people_vaccinated_2nd_dose'],
    value = ['cases', 'hospital','deaths', 'No_of_people_vaccinated_1st_dose', 'No_of_people_vaccinated_2nd_dose'],
    rows=5,
    description= 'Category:',
    disable=False)

df4_interactive_scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Type of graph:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.VBox([df4_interactive_series, df4_interactive_scale])

def df4_interactive_graph(gcols, gscale):
    ncols = len(gcols)
  
    if gscale == 'linear':
        logscale = False
        if ncols > 0:
            df4[list(gcols)].plot()
            plt.show()
        else:
            print("Please click one of the interactable buttons")
            print("If you would like to select more than one category, hold down tab and select with your mouse")
    else:
        logscale = True
        if ncols > 0:
            df4[list(gcols)].plot(logy=logscale)
            plt.show()
        else:
            print("Please click one of the interactable buttons")
            print("If you would like to select more than one category, hold down tab and select with your mouse")

            

graph_4 = wdg.interactive_output(df4_interactive_graph, {'gcols': df4_interactive_series, 'gscale': df4_interactive_scale})

def refresh_graph(button):
    current=df4_interactive_scale.value
    if current == df4_interactive_scale.options[0]:
        other= df4_interactive_scale.options[1]
    else:
        other= df4_interactive_scale.options[0]
    df4_interactive_scale.value=other
    df4_interactive_scale.value=current

refresh= wdg.Button(
    description='refresh data',
    disabled=False,
    button_style='success',
    tooltip= 'click to refresh data in graph',
    icon= 'arrows'
)
#I decided to not make the button one click only as users may want to refresh data for a graph more than once.
refresh.on_click(refresh_graph)

display(controls,refresh, graph_4)



df5_interactive_series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths', 'No_of_people_vaccinated_1st_dose', 'No_of_people_vaccinated_2nd_dose'], 
    value=['cases', 'hospital', 'deaths', 'No_of_people_vaccinated_1st_dose', 'No_of_people_vaccinated_2nd_dose'], 
    rows=5,
    description= 'Category: ',
    disabled=False
)
df5_interactive_scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Type of graph:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.VBox([df5_interactive_series, df5_interactive_scale])

def df5_interactive_graph(gcols, gscale):
    ncols = len(gcols)
  
    if gscale == 'linear':
        logscale = False
        if ncols > 0:
            df5[list(gcols)].plot()
            plt.show()
        else:
            print("Please click one of the interactable buttons")
            print("If you would like to select more than one category, hold down tab and select with your mouse")
    else:
        logscale = True
        if ncols > 0:
            df5[list(gcols)].plot(logy=logscale)
            plt.show()
        else:
            print("Please click one of the interactable buttons")
            print("If you would like to select more than one category, hold down tab and select with your mouse")

            

graph_5 = wdg.interactive_output(df5_interactive_graph, {'gcols': df5_interactive_series, 'gscale': df5_interactive_scale})

def refresh_graph(button):
    current=df5_interactive_scale.value
    if current == df5_interactive_scale.options[0]:
        other= df5_interactive_scale.options[1]
    else:
        other= df5_interactive_scale.options[0]
    df5_interactive_scale.value=other
    df5_interactive_scale.value=current

refresh= wdg.Button(
    description='refresh data',
    disabled=False,
    button_style='success',
    tooltip= 'click to refresh data in graph',
    icon= 'arrows'
)
#I decided to not make the button one click only as users may want to refresh data for a graph more than once.
refresh.on_click(refresh_graph)

display(controls,refresh, graph_5)

#hospital category returns data has no postive values when selecting the log scaled button, but note it can be graphed as a linear graph (please select linear radio button). The reason for this is when retrieving the data from the API, it appears hospital admissions are not entered regionally. This is an issue with the data which is outside of the remit of this coursework and not the code. 

df6_interactive_series=wdg.SelectMultiple(
    options =['cases', 'hospital', 'occupied_ventilators','deaths'],
    value = ['cases', 'hospital', 'occupied_ventilators','deaths'],
    rows=4,
    description= 'Category:',
    disable=False)

df6_interactive_scale=wdg.RadioButtons(
    options=['log', 'linear'],
#   value='pineapple', # Defaults to 'pineapple'
#   layout={'width': 'max-content'}, # If the items' names are long
    description='Type of graph:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.VBox([df6_interactive_series, df6_interactive_scale])

def df6_interactive_graph(gcols, gscale):
    ncols = len(gcols)
    if gscale == 'linear':
        logscale = False
        if ncols > 0:
            df6[list(gcols)].plot()
            plt.show()
        else:
            print("Please click one of the interactable buttons")
            print("If you would like to select more than one category, hold down tab and select with your mouse")
    else:
        logscale = True
        if ncols > 0:
            df6[list(gcols)].plot(logy=logscale)
            plt.show()
        else:
            print("Please click one of the interactable buttons")
            print("If you would like to select more than one category, hold down tab and select with your mouse")

graph_6 = wdg.interactive_output(df6_interactive_graph, {'gcols': df6_interactive_series, 'gscale': df6_interactive_scale})

def refresh_graph(button):
    current=df6_interactive_scale.value
    if current == df6_interactive_scale.options[0]:
        other= df6_interactive_scale.options[1]
    else:
        other= df6_interactive_scale.options[0]
    df6_interactive_scale.value=other
    df6_interactive_scale.value=current

refresh= wdg.Button(
    description='refresh data',
    disabled=False,
    button_style='success',
    tooltip= 'click to refresh data in graph',
    icon= 'arrows'
)
#I decided to not make the button one click only as users may want to refresh data for a graph more than once.
refresh.on_click(refresh_graph)

display(controls,refresh, graph_6)

Button(button_style='success', description='refresh graph', icon='arrows', style=ButtonStyle(), tooltip='click…

Output()

Button(button_style='success', description='refresh graph', icon='arrows', style=ButtonStyle(), tooltip='click…

Output()

Button(button_style='success', description='refresh data', icon='arrows', style=ButtonStyle(), tooltip='click …

...

Output()

Button(button_style='success', description='refresh data', icon='arrows', style=ButtonStyle(), tooltip='click …

Output()

Button(button_style='success', description='refresh data', icon='arrows', style=ButtonStyle(), tooltip='click …

Output()